In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.dataset_getters.pytorch_datasets import HumanEnhancersCohn
from genomic_benchmarks.models.torch_cnn import CNN
from utils import coll_factory, LetterTokenizer, build_vocab, check_seq_lengths, check_config

## Config

In [3]:
config = {
    "use_padding": False,
    "run_on_gpu": True,
    "dataset": HumanEnhancersCohn,
    "number_of_classes": 1,
    "dataset_version": 0,
    "force_download": False,
    "epochs": 15,
    "embedding_dim": 100,
    "batch_size": 32,
}
check_config(config)

config is correct


## Choose the dataset

In [4]:
get_dataset_fn = config["dataset"]
train_dset = get_dataset_fn('train', force_download=config["force_download"], version=config["dataset_version"])

Unzipping...Done.


## Tokenizer and vocab

In [5]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 6
{'<eos>': 5, 'G': 4, 'A': 3, 'C': 2, 'T': 1, '<bos>': 0}


## Dataloader and batch preparation

In [6]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_seq_len, nn_input_len = check_seq_lengths(dataset=train_dset, config=config)

# Data Loader
if(config["use_padding"]):
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = nn_input_len)
else:
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = None)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cuda device
max_seq_len  500


## Model

In [7]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=nn_input_len
).to(device)

/opt/conda/envs/bench_env/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


## Training

In [8]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0
Train metrics: 
 Accuracy: 66.5%, Avg loss: 0.644216 

Epoch 1
Train metrics: 
 Accuracy: 67.0%, Avg loss: 0.637457 

Epoch 2
Train metrics: 
 Accuracy: 68.9%, Avg loss: 0.636182 

Epoch 3
Train metrics: 
 Accuracy: 69.8%, Avg loss: 0.634578 

Epoch 4
Train metrics: 
 Accuracy: 67.6%, Avg loss: 0.634644 

Epoch 5
Train metrics: 
 Accuracy: 69.0%, Avg loss: 0.630810 

Epoch 6
Train metrics: 
 Accuracy: 71.3%, Avg loss: 0.631119 

Epoch 7
Train metrics: 
 Accuracy: 70.8%, Avg loss: 0.627720 

Epoch 8
Train metrics: 
 Accuracy: 72.4%, Avg loss: 0.623563 

Epoch 9
Train metrics: 
 Accuracy: 68.9%, Avg loss: 0.631351 

Epoch 10
Train metrics: 
 Accuracy: 73.3%, Avg loss: 0.623932 

Epoch 11
Train metrics: 
 Accuracy: 72.6%, Avg loss: 0.623998 

Epoch 12
Train metrics: 
 Accuracy: 72.2%, Avg loss: 0.620767 

Epoch 13
Train metrics: 
 Accuracy: 72.6%, Avg loss: 0.620684 

Epoch 14
Train metrics: 
 Accuracy: 73.7%, Avg loss: 0.618135 



## Testing

In [9]:
test_dset = get_dataset_fn('test', force_download=config["force_download"], version=config["dataset_version"])
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

model.test(test_loader)

Unzipping...Done.
test_loss  139.58674901723862
num_batches 218
correct 4834
size 6948
Test Error: 
 Accuracy: 69.6%, Avg loss: 0.640306 

